### To setup the environment for downloading the dataset and the YOLOv5 model.

In [1]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt
!pip install roboflow

import torch

from roboflow import Roboflow
rf = Roboflow(api_key="RtdzCZNW8mDA4ilAU416")
project = rf.workspace("projects-1vokb").project("frogs-ft5yx")
dataset = project.version(1).download("yolov5")

Cloning into 'yolov5'...
remote: Enumerating objects: 15365, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15365 (delta 1), reused 3 (delta 0), pack-reused 15355
Receiving objects: 100% (15365/15365), 14.30 MiB | 3.82 MiB/s, done.
Resolving deltas: 100% (10505/10505), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 4.7 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 16.4 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 18.5 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.4 MB/s eta 0:00:0031m29.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=3d07c014f7da0660e71deb652aee41a76e343c74e1f9a372a44ea58353ab01d2
  Stored in directory: /home/pptr/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.3
    Uninstalling idna-3.3:
      Successfull

Extracting Dataset Version Zip to frogs-1 in yolov5pytorch::  43%|█████████████████████████████▏                                     | 333/766 [00:00<00:00, 3324.51it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Train on polygons

In [ ]:
!python yolov5/segment/train.py --data frogs-1/data.yaml --img 320 --epochs 10000 --batch 128 --name trained-seg

**Convert the polygon test set from polygon to bounding box coordinates.**

In [4]:
import numpy as np
import os
import cv2

def convert_to_bounding_box(segmentation_label):
    x_coord, y_coord = [], []
    for i in range(0, len(segmentation_label), 2):
        x_coord.append(segmentation_label[i])
        y_coord.append(segmentation_label[i+1])
    return convert_to_yolov5(min(x_coord), min(y_coord), max(x_coord), max(y_coord))

def convert_to_yolov5(xmin, ymin, xmax, ymax):
    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin
    return np.array((x_center, y_center, width, height))


def from_xmin_ymin_xmax_ymax_to_yolov5(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()
            idx_to_delete = set()
            for i, line in enumerate(lines):
                values = line.split()
                if len(values) < 6:
                    idx_to_delete.add(i)
                    continue
                class_label = values[0]
                polygon = [float(val) for val in values[1:]]
                bbox = convert_to_bounding_box(polygon)
                lines[i] = f"{class_label} {' '.join(map(str, bbox))}"  + "\n"
            new_lines = []
            for idx, line in enumerate(lines):
                if idx not in idx_to_delete:
                    new_lines.append(line)
            with open(file_path, 'w') as file:
                file.writelines(new_lines)
gt_folder = "frogs-1/test/labels/" # ground truth polygons
from_xmin_ymin_xmax_ymax_to_yolov5(gt_folder)

### Evaluate on test set

In [17]:
!python yolov5/val.py --weights yolov5/runs/train-seg/trained-seg2/weights/best.pt --img 320 --batch 128 --data frogs-1/data.yaml

val: data=frogs-1/data.yaml, weights=['yolov5/runs/train-seg/trained-seg2/weights/best.pt'], batch_size=128, imgsz=320, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-133-gcca5e21 Python-3.9.13 torch-2.0.0+cu117 CPU

Fusing layers... 
Model summary: 165 layers, 7414604 parameters, 0 gradients, 25.7 GFLOPs
val: Scanning /home/pptr/Desktop/roboflow/polygons-vs-bbox/frogs-1/test/labels.c
                 Class     Images  Instances          P          R      mAP50   
                   all         39         40      0.531      0.615      0.589      0.465
  alytes obstetricians         39          3      0.353      0.563      0.356      0.248
             bufo bufo         39          1      0.478          1      0.497      0.448
         bufo calamita         

# Train on bounding box

**Convert the training set labels from polygons to bounding box coordinates.**

In [ ]:
training_folder = "frogs-1/train/labels/" # ground truth polygons
from_xmin_ymin_xmax_ymax_to_yolov5(training_folder)

### Train

In [ ]:
!python yolov5/train.py --data frogs-1/data.yaml --img 320 --epochs 10000 --batch 128 --name trained-od

### Evaluate on test set

In [3]:
!python yolov5/val.py --weights yolov5/runs/train/trained-od/weights/best.pt --img 320 --batch 128 --data frogs-1/data.yaml

val: data=frogs-1/data.yaml, weights=['yolov5/runs/train/trained-od/weights/best.pt'], batch_size=128, imgsz=320, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-133-gcca5e21 Python-3.9.13 torch-2.0.0+cu117 CPU

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /home/pptr/Desktop/roboflow/polygons-vs-bbox/frogs-1/test/labels.c
                 Class     Images  Instances          P          R      mAP50   
                   all         39         40      0.569      0.761      0.779      0.625
  alytes obstetricians         39          3      0.418      0.667      0.806      0.608
             bufo bufo         39          1       0.16          1      0.995      0.895
         bufo calamita         39    